In [29]:
## RANDOM FOREST FROM SCRATCH ##


class RandomForest():      # define class of RANDOMFOREST
    def __init__(self, x, y, n_trees, n_features, sample_sz, depth=10, min_leaf=5): # init 
        np.random.seed(12)     # setting a random seed
        if n_features == 'sqrt':   # condition: if given n_features are refered as squareroot then
            self.n_features = int(np.sqrt(x.shape[1])) # calculate squareroot of col number of x
        elif n_features == 'log2': # condition 2: if refered to log2 then
            self.n_features = int(np.log2(x.shape[1]))  # calculate log2 of column number of x
        else:
            self.n_features = n_features   # if no condition given then do nothing
        print(self.n_features, "sha: ",x.shape[1])  # print number of columns of x   
        # write in refered data to self-variables
        self.x, self.y, self.sample_sz, self.depth, self.min_leaf  = x, y, sample_sz, depth, min_leaf
        self.trees = [self.create_tree() for i in range(n_trees)] # refer create tree in range of n_trees
        
        # sample_size: the number of rows randomly selected and passed onto each tree. This is usually equal 
        # to total number of rows but can be reduced to increase performance and decrease correlation of trees 
        # in some cases (bagging of trees is a completely separate machine learning technique)
    
    def create_tree(self):  # function to create a decision tree
        # create random index permutation of sample size, depending on y-length  
        idxs = np.random.permutation(len(self.y))[:self.sample_sz] 
        # random permutation of sample size in the length of y
        # indxs: this parameter exists to keep track of which indices of the original set goes to the right 
        # and which goes to the left tree. Hence every tree has this parameter “indxs” which stores the indices 
        # of the rows it contains. Prediction is made by averaging these rows.
        f_idxs = np.random.permutation(self.x.shape[1])[:self.n_features] # permutation of columns (features)
        # for the x column length
        # return decisiontree of permuted indexes of x and y with given n-features, permuted columns, 
        # and an np.array in the range of sample size, given depth of DT and min number of leafs
        return DecisionTree(self.x.iloc[idxs], self.y[idxs], self.n_features, f_idxs,
                    idxs=np.array(range(self.sample_sz)),depth = self.depth, min_leaf=self.min_leaf)
        
        # min_leaf: minimum number of rows required in a node to cause further split. Lower the min_leaf, 
        # higher the depth of the tree.
        
        
        
        
    def predict(self, x):    # function to predict x_pred of x (), avaraging of predictions
        return np.mean([t.predict(x) for t in self.trees], axis=0)

def std_agg(cnt, s1, s2): return math.sqrt((s2/cnt) - (s1/cnt)**2) # function to calculate standard deviation
                                                                   # 
class DecisionTree():    # class for decision tree
    
    def __init__(self, x, y, n_features, f_idxs,idxs,depth=10, min_leaf=5): # init with defaults
        # write in given data and properties
        self.x, self.y, self.idxs, self.min_leaf, self.f_idxs = x, y, idxs, min_leaf, f_idxs
        self.depth = depth
        print(f_idxs)
#         print(self.depth)
        self.n_features = n_features
        self.n, self.c = len(idxs), x.shape[1]   # n = length of permuted indices and c is number of cols of x
        self.val = np.mean(y[idxs])              # val is mean of y with permuted indices
        self.score = float('inf')                # score = float for infinit 
        self.find_varsplit()                     # refering the next function
        
    def find_varsplit(self):                     # function varsplit
        for i in self.f_idxs: self.find_better_split(i) # looping through find_better_split
        if self.is_leaf: return                         
        x = self.split_col                       # write vals of split_col to x
        lhs = np.nonzero(x<=self.split)[0]       # write indices for non-zero values of x <= split to lhs
        rhs = np.nonzero(x>self.split)[0]        # write indices for non-zero values of x > split to rhs
        # rhs stores the indices of rows passed onto the right decision tree.
        # lhs to the left decision tree
        
        # random permutations in range of cols of x  
        lf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        rf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        self.lhs = DecisionTree(self.x, self.y, self.n_features, lf_idxs, self.idxs[lhs], 
                                depth=self.depth-1, min_leaf=self.min_leaf)
        self.rhs = DecisionTree(self.x, self.y, self.n_features, rf_idxs, self.idxs[rhs], 
                                depth=self.depth-1, min_leaf=self.min_leaf)

    def find_better_split(self, var_idx):    # function to find a better data split
        x, y = self.x.values[self.idxs,var_idx], self.y[self.idxs]
        sort_idx = np.argsort(x)             # save sorted indices of x
        sort_y,sort_x = y[sort_idx], x[sort_idx] # choose y's for sorted indices of x
        # set rhs_cnt to length of random permuted indices n, rhs_sum to sum of sorted y's and sum squared
        rhs_cnt,rhs_sum,rhs_sum2 = self.n, sort_y.sum(), (sort_y**2).sum()
        # set lhs count and lhs sums to zeros
        lhs_cnt,lhs_sum,lhs_sum2 = 0,0.,0.

        for i in range(0,self.n-self.min_leaf-1): # for loop in range of zero to n-permuted indices minus 
                                                  # minimum number of leafes minus 1
            xi,yi = sort_x[i],sort_y[i]         
            lhs_cnt += 1; rhs_cnt -= 1            # sum up lhs_cnt by 1 and rhs_cnt held at zero by -1
            lhs_sum += yi; rhs_sum -= yi          # same for sum
            lhs_sum2 += yi**2; rhs_sum2 -= yi**2  # same for sum squared
            if i<self.min_leaf or xi==sort_x[i+1]:# condition continue if i<minimum leaf number or if 
                                                  # xi equal sorted x at i+1
                continue

            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2) # calc standard deviations for left decision tree
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2) # same for right decision tree
            # current score = the split score for each iteration is simply the weighted average of standard 
            # deviation of the two halves with number of rows in each half as their weights. Lower score 
            # facilitates lower variance, lower variance facilitates grouping of similar data which will 
            # lead to better predictions.
            curr_score = lhs_std*lhs_cnt + rhs_std*rhs_cnt
            if curr_score<self.score: 
                self.var_idx,self.score,self.split = var_idx,curr_score,xi
            # whenever the current score is better (less than the currently saved score in self.score) we will 
            # update the current score and store this column in the variable self.var_idx (remember this is the 
            # variable that helps selecting the column for two of our property decorators) and save the value 
            # upon which the split is made, in the variable self.split

    @property
    def split_name(self): return self.x.columns[self.var_idx]
    #A property decorator to return the name of the column we’re splitting over. var_idx is the index of this 
    # column, we will calculate this index in the find_better_split function along with value of the column we 
    # split upon
    
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]
    # A property decorator to return the column at index var_idx with elements at indices given by indxs variable. 
    # Basically, segregates a column with selected rows.
    
    @property
    def is_leaf(self): return self.score == float('inf') or self.depth <= 0 
    # A leaf node is the one that has never made a split thus it has a score of infinite (as mentioned above) 
    # hence this functions is used to identify leaf nodes. Also in case we’ve crossed maximum depth i.e. 
    # self.depth <= 0, it’s a leaf node as we can’t go any deeper.

    def predict(self, x):   
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx]<=self.split else self.rhs
        return t.predict_row(xi)

https://towardsdatascience.com/random-forests-and-decision-trees-from-scratch-in-python-3e4fa5ae4249

In [31]:
float('inf')

inf

In [9]:
import numpy as np
x = np.array([1,3,9])
np.argsort(x)

array([0, 1, 2])